# UAVSAR Flood
## 1. Get UAVSAR Data
## 2. Convert to Geotiff & Subset with GDAL

Sept 17: https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=peedee_15100_18064_002_180917_L090_CX_01#data 

Sept 23: https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=peedee_15100_18069_007_180923_L090_CX_01#data 

Coherence: https://uavsar.jpl.nasa.gov/cgi-bin/product.pl?jobName=peedee_15100_18067-005_18068-005_0002d_s01_L090_01#data 

exec bash (set to bash)

for i in *.grd; do gdal_translate -projwin -79.50 34.042 -79.3507 33.9462 $i $i.tif; done

gdal_translate -projwin -79.50 34.042 -79.3507 33.9462 coherence.tif coherence_subset.tif


In [ ]:
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y


In [ ]:
import gdal

1. Download Coherence File and Annotation file
2. Make .hdr file for .cor.grd (need to change samples, lines, and left corner lat/lon)
3. Run through Naiara's script 'raster_read.py' to generate a GeoTIFF from the .hdr and .cor.grd files
4. This GeoTIFF and the Pauli GeoTIFFs have a black outline in areas of no data. Need to get rid of this:
5. In coherence file, set 0 as Nodata

gdal_translate -of GTiff -a_nodata 0 -a_srs EPSG:4326 peedee_15100_18064-002_18069-007_0006d_s01_L090HH_01.cor.grd.tif peedee_15100_18064-002_18069-007_0006d_s01_L090HH_01_nodata.cor.grd.tif 

6. Now set nodata to -999 and only grab values above this

gdal_calc.py -A peedee_15100_18064-002_18069-007_0006d_s01_L090HH_01_nodata.cor.grd.tif --outfile out.tif --NoDataValue -999 --calc="1*(A>-999)”

7. Create polygon of this new subset

gdal_polygonize.py -8 -b 1 -f "ESRI Shapefile" out.tif pols

8. Crop Pauli tifs to this extent. 

gdalwarp -of GTiff -cutline pols/out.shp -crop_to_cutline -dstalpha peedee_15100_18064_002_180917_L090_CX_01_pauli.tif peedee_15100_18064_002_180917_L090_CX_01_pauli_cropped.tif
gdalwarp -of GTiff -cutline pols/out.shp -crop_to_cutline -dstalpha peedee_15100_18065_010_180918_L090_CX_01_pauli.tif peedee_15100_18065_010_180918_L090_CX_01_pauli_cropped.tif
gdalwarp -of GTiff -cutline pols/out.shp -crop_to_cutline -dstalpha peedee_15100_18067_005_180920_L090_CX_01_pauli.tif peedee_15100_18067_005_180920_L090_CX_01_pauli_cropped.tif
gdalwarp -of GTiff -cutline pols/out.shp -crop_to_cutline -dstalpha peedee_15100_18068_005_180922_L090_CX_01_pauli.tif peedee_15100_18068_005_180922_L090_CX_01_pauli_cropped.tif
gdalwarp -of GTiff -cutline pols/out.shp -crop_to_cutline -dstalpha peedee_15100_18069_007_180923_L090_CX_01_pauli.tif peedee_15100_18069_007_180923_L090_CX_01_pauli_cropped.tif


######### Make Map Tiles #############
python
Import gdal2tiles

gdal2tiles.generate_tiles("peedee_15100_18069_007_180923_L090_CX_01_pauli_cropped.tif", "./peedee23", zoom='5-14')
gdal2tiles.generate_tiles("peedee_15100_18068_005_180922_L090_CX_01_pauli_cropped.tif", "./peedee22", zoom='5-14')
gdal2tiles.generate_tiles("peedee_15100_18067_005_180920_L090_CX_01_pauli_cropped.tif", "./peedee20", zoom='5-14')
gdal2tiles.generate_tiles("peedee_15100_18065_010_180918_L090_CX_01_pauli_cropped.tif", "./peedee18", zoom='5-14')
gdal2tiles.generate_tiles("peedee_15100_18064_002_180917_L090_CX_01_pauli_cropped.tif", "./peedee17", zoom='5-14')


######## Make PNGs for Slideshow #############

gdal_translate -of PNG -ot Byte peedee_15100_18069_007_180923_L090_CX_01_pauli_cropped.tif peedee_15100_180923.png
gdal_translate -of PNG -ot Byte peedee_15100_18068_005_180922_L090_CX_01_pauli_cropped.tif peedee_15100_180922.png
gdal_translate -of PNG -ot Byte peedee_15100_18067_005_180920_L090_CX_01_pauli_cropped.tif peedee_15100_180920.png
gdal_translate -of PNG -ot Byte peedee_15100_18065_010_180918_L090_CX_01_pauli_cropped.tif peedee_15100_180918.png
gdal_translate -of PNG -ot Byte peedee_15100_18064_002_180917_L090_CX_01_pauli_cropped.tif peedee_15100_180917.png

#hdr_commands.sh
python /Users/peacock/Desktop/UAVSAR_Quicklooks/BuildUAVSARhdr.py -i peedee_15100_18064_002_180917_L090_CX_01.ann.txt -r 
peedee_15100_18064_002_180917_L090HHHH_CX_01.grd -p HHHH

python /Users/peacock/Desktop/UAVSAR_Quicklooks/BuildUAVSARhdr.py -i peedee_15100_18064_002_180917_L090_CX_01.ann.txt -r peedee_15100_18064_002_180917_L090HVHV_CX_01.grd -p HVHV


python /Users/peacock/Desktop/UAVSAR_Quicklooks/BuildUAVSARhdr.py -i peedee_15100_18064_002_180917_L090_CX_01.ann.txt -r peedee_15100_18064_002_180917_L090VVVV_CX_01.grd -p VVVV

python /Users/peacock/Desktop/UAVSAR_Quicklooks/BuildUAVSARhdr.py -i peedee_15100_18069_007_180923_L090_CX_01.ann.txt -r peedee_15100_18069_007_180923_L090HHHH_CX_01.grd -p HHHH

python /Users/peacock/Desktop/UAVSAR_Quicklooks/BuildUAVSARhdr.py -i peedee_15100_18069_007_180923_L090_CX_01.ann.txt -r peedee_15100_18069_007_180923_L090HVHV_CX_01.grd -p HVHV

python /Users/peacock/Desktop/UAVSAR_Quicklooks/BuildUAVSARhdr.py -i peedee_15100_18069_007_180923_L090_CX_01.ann.txt -r peedee_15100_18069_007_180923_L090VVVV_CX_01.grd -p VVVV
________________________

python /Users/peacock/Desktop/UAVSAR_Quicklooks/raster_read.py -i peedee_15100_18064_002_180917_L090HHHH_CX_01.grd

python /Users/peacock/Desktop/UAVSAR_Quicklooks/raster_read.py -i peedee_15100_18064_002_180917_L090HVHV_CX_01.grd

python /Users/peacock/Desktop/UAVSAR_Quicklooks/raster_read.py -i peedee_15100_18064_002_180917_L090VVVV_CX_01.grd

python /Users/peacock/Desktop/UAVSAR_Quicklooks/raster_read.py -i peedee_15100_18069_007_180923_L090HHHH_CX_01.grd

python /Users/peacock/Desktop/UAVSAR_Quicklooks/raster_read.py -i peedee_15100_18069_007_180923_L090HVHV_CX_01.grd

python /Users/peacock/Desktop/UAVSAR_Quicklooks/raster_read.py -i peedee_15100_18069_007_180923_L090VVVV_CX_01.grd
_______________________________

gdal_translate peedee_15100_18064_002_180917_L090HHHH_CX_01.grd.tif peedee_15100_18064_002_180917_L090HHHH_CX_01_cropped.grd.tif -projwin -79.5524 34.0554 -79.3462 33.8931

gdal_translate peedee_15100_18064_002_180917_L090HVHV_CX_01.grd.tif peedee_15100_18064_002_180917_L090HVHV_CX_01_cropped.grd.tif -projwin -79.5524 34.0554 -79.3462 33.8931

gdal_translate peedee_15100_18064_002_180917_L090VVVV_CX_01.grd.tif peedee_15100_18064_002_180917_L090VVVV_CX_01_cropped.grd.tif -projwin -79.5524 34.0554 -79.3462 33.8931

gdal_translate peedee_15100_18069_007_180923_L090HHHH_CX_01.grd.tif peedee_15100_18069_007_180923_L090HHHH_CX_01_cropped.grd.tif -projwin -79.5524 34.0554 -79.3462 33.8931

gdal_translate peedee_15100_18069_007_180923_L090HVHV_CX_01.grd.tif peedee_15100_18069_007_180923_L090HVHV_CX_01_cropped.grd.tif -projwin -79.5524 34.0554 -79.3462 33.8931

gdal_translate peedee_15100_18069_007_180923_L090VVVV_CX_01.grd.tif peedee_15100_18069_007_180923_L090VVVV_CX_01_cropped.grd.tif -projwin -79.5524 34.0554 -79.3462 33.8931

gdal_translate coherence.tif coherence_cropped.tif -projwin -79.5524 34.0554 -79.3462 33.8931
